# Physics 192.1 Experiment 4 Set A

We determine the groove spacing of given CDs and DVDs using the grating equation [[1](https://en.wikipedia.org/wiki/Diffraction_grating#Theory_of_operation)]

$$
    d \sin \theta = n \lambda
$$

where 
- $d$ is the spacing between the slits
- $\theta$ is the $n$-th order diffraction angle
- $n$ is the order of diffraction $n = 1,2,3 ...$
- $\lambda$ is the wavelength of light



In [37]:
# ----------------------------------- #
#              IMPORTS
# ----------------------------------- #

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

# for aesthetic plots
def science_plot(fontsize=9, scistyle=True, show_latex=True):
    # Default settings (applied to both 2D and 3D)
    if scistyle:
        import scienceplots
        plt.style.use(['science','grid','notebook'])
    if show_latex:
        plt.rcParams.update({
            # Latex Use
            'text.usetex'     : True,        # Use LaTeX for text rendering
            'font.family'     : 'serif',     # Set font family to serif
        })

    plt.rcParams.update({
        # Fontsizes
        'font.size'       : fontsize,    # General font size
        'axes.titlesize'  : fontsize,    # Font size of the axes title
        'axes.labelsize'  : fontsize,    # Font size of the axes labels
        'xtick.labelsize' : fontsize,    # Font size of the x-axis tick labels
        'ytick.labelsize' : fontsize,    # Font size of the y-axis tick labels
        'legend.fontsize' : fontsize,    # Font size of the legend
        'figure.titlesize': fontsize,    # Font size of the figure title

        # Legend
        'legend.fancybox' : False,       # Disable the fancy box for legend
        'legend.edgecolor': 'k',         # Set legend border color to black
    })

    def optional():
        # # Grid settings
        # "grid.linestyle": "--",
        # "grid.color": "gray",
        # "grid.linewidth": 1,
        # "axes.grid": True,

        # # Minor grid (default, but may be overridden for 3D)
        # "xtick.minor.visible": True,
        # "ytick.minor.visible": True,

        # # Tick settings (both major & minor)
        # "xtick.direction": "in",
        # "ytick.direction": "in",
        # "xtick.top": True,
        # "xtick.bottom": True,
        # "ytick.left": True,
        # "ytick.right": True,

        # 'colorbar.ticks.direction': 'out',

        # # Major ticks
        # "xtick.major.width": 1,
        # "ytick.major.width": 1,
        # "xtick.major.size": 5,
        # "ytick.major.size": 5,

        # # Minor ticks
        # "xtick.minor.width": 1,
        # "ytick.minor.width": 1,
        # "xtick.minor.size": 2.5,
        # "ytick.minor.size": 2.5,

        # # Spine (border) width
        # "axes.linewidth": 1
        pass 

science_plot(fontsize=9)

Here we have the raw data, with `distance` being the distance of the disc from the wall, `L1` and `R1` showing the distance of the left and right fringes from the central beam, and the number being the order of the fringe (`R2`). All units are in $\text{cm}$, with an uncertainty of $\pm 0.1 \text{ cm}$

In [38]:
df = pd.read_csv("data.csv")
print(df)

  type  distance    R1    R2    L1    L2
0  DVD      55.1  90.8   NaN  90.8   NaN
1  DVD      27.5  45.4   NaN  45.2   NaN
2  DVD      29.0  47.8   NaN  47.5   NaN
3  DVD      19.2  31.9   NaN  31.4   NaN
4  DVD      11.1  18.2   NaN  18.1   NaN
5   CD      69.3  29.2  84.9  29.8  89.4
6   CD      38.1  16.4  48.4  16.1  47.5
7   CD      38.0  16.2  48.5  16.2  46.7
8   CD      40.3  17.2  50.0  17.2  50.7
9   CD      18.5   7.9  23.1   7.9  23.0


We get the diffraction angle $\theta$ using the arctangent function

$$
    \theta = \tan^{-1} \left( \frac{\delta}{\ell} \right)
$$

Where $\delta$ is the distance of the difractor from the wall and $\ell$ being the distance of the fringe from the central beam

In [39]:
# Angles in radians 
df['thetaR1'] = np.arctan(df['R1'] / df['distance'])
df['thetaR2'] = np.arctan(df['R2'] / df['distance'])
df['thetaL1'] = np.arctan(df['L1'] / df['distance'])
df['thetaL2'] = np.arctan(df['L2'] / df['distance'])
# Angles in degrees
thetaR1deg = np.rad2deg(df['thetaR1'])
thetaR2deg = np.rad2deg(df['thetaR2'])
thetaL1deg = np.rad2deg(df['thetaL1'])
thetaL2deg = np.rad2deg(df['thetaL2'])

deg_df = pd.DataFrame({
    'thetaR1 deg': thetaR1deg,
    'thetaR2 deg': thetaR2deg,
    'thetaL1 deg': thetaL1deg,
    'thetaL2 deg': thetaL2deg
})
print(deg_df)

   thetaR1 deg  thetaR2 deg  thetaL1 deg  thetaL2 deg
0    58.749443          NaN    58.749443          NaN
1    58.795584          NaN    58.683373          NaN
2    58.755039          NaN    58.594836          NaN
3    58.957069          NaN    58.555675          NaN
4    58.621398          NaN    58.480890          NaN
5    22.848437    50.776805    23.268403    52.218310
6    23.289277    51.790530    22.907567    51.266717
7    23.089317    51.921118    23.089317    50.864596
8    23.112741    51.131186    23.112741    51.519802
9    23.123792    51.309957    23.123792    51.188616


The equipment used was a $\text{He-Ne}$ laser, which has a known light wavelength of $\lambda = 632.8 \text{ nm}$

Using that, we can solve for the grating spacing using 
$$
    d = \frac{n\lambda}{\sin\theta}
$$

We group the fringes by order, then take the average of each row.

In [43]:
# Wavelength in cm
lambda_cm = 632.8e-7  # 632.8 nm = 6.328e-5 cm

## All units are in cm measure
# Grating spacing d = m* lambda / sin theta
order_map = {'thetaR1':1, 'thetaL1':1, 'thetaR2':2, 'thetaL2':2}
for ang_col, n in order_map.items():
    dcol = f'd_{ang_col}'
    df[dcol] = n * lambda_cm / np.sin(df[ang_col])

# Row averages
d_cols = [c for c in df.columns if c.startswith('d_')]
df['d_row_mean'] = df[d_cols].mean(axis=1, skipna=True)

pd.set_option('display.float_format', '{:.8f}'.format)
print(df[d_cols])

   d_thetaR1  d_thetaL1  d_thetaR2  d_thetaL2  d_row_mean
0 0.00007402 0.00007402        NaN        NaN  0.00007402
1 0.00007398 0.00007407        NaN        NaN  0.00007403
2 0.00007402 0.00007414        NaN        NaN  0.00007408
3 0.00007386 0.00007417        NaN        NaN  0.00007402
4 0.00007412 0.00007423        NaN        NaN  0.00007418
5 0.00016297 0.00016019 0.00016337 0.00016013  0.00016166
6 0.00016005 0.00016257 0.00016107 0.00016224  0.00016148
7 0.00016136 0.00016136 0.00016078 0.00016316  0.00016167
8 0.00016121 0.00016121 0.00016255 0.00016167  0.00016166
9 0.00016113 0.00016113 0.00016214 0.00016242  0.00016171


We take the average of the result by type

In [ ]:
print(df[['type'] + d_cols + ['d_row_mean']])
# Summary per type
summary = (df.groupby('type')['d_row_mean']
             .agg(['mean','std','count'])
             .rename(columns={'mean':'d_mean_cm','std':'d_std_cm'}))
print(" ")
print(summary)

  type  d_thetaR1  d_thetaL1  d_thetaR2  d_thetaL2  d_row_mean  d_row_mean
0  DVD 0.00007402 0.00007402        NaN        NaN  0.00007402  0.00007402
1  DVD 0.00007398 0.00007407        NaN        NaN  0.00007403  0.00007403
2  DVD 0.00007402 0.00007414        NaN        NaN  0.00007408  0.00007408
3  DVD 0.00007386 0.00007417        NaN        NaN  0.00007402  0.00007402
4  DVD 0.00007412 0.00007423        NaN        NaN  0.00007418  0.00007418
5   CD 0.00016297 0.00016019 0.00016337 0.00016013  0.00016166  0.00016166
6   CD 0.00016005 0.00016257 0.00016107 0.00016224  0.00016148  0.00016148
7   CD 0.00016136 0.00016136 0.00016078 0.00016316  0.00016167  0.00016167
8   CD 0.00016121 0.00016121 0.00016255 0.00016167  0.00016166  0.00016166
9   CD 0.00016113 0.00016113 0.00016214 0.00016242  0.00016171  0.00016171
      d_mean_cm   d_std_cm  count
type                             
CD   0.00016164 0.00000009      5
DVD  0.00007406 0.00000007      5


We transform the data to $\text{nm}$, and take the $\text{SDOM}$ for the uncertainty.

We also calculate the grooves per $\text{mm}$ of the result (taken using $0.1 / d$)

In [ ]:
# Reset pandas display to default
pd.reset_option('display.float_format')

# --- Groove spacing in nm ---
summary['d_mean_nm'] = summary['d_mean_cm'] * 1e7 # cm --> nm
summary['d_std_nm'] = summary['d_std_cm'] * 1e7

# sdom for d (in nm)
summary['d_sdom_nm'] = summary['d_std_nm'] / np.sqrt(summary['count'])

# --- Grooves per mm ---
summary['grooves_per_mm'] = 0.1 / summary['d_mean_cm']

# print data
print(summary[['d_mean_nm','d_sdom_nm','d_std_nm','grooves_per_mm']])

        d_mean_nm  d_sdom_nm  d_std_nm  grooves_per_mm
type                                                  
CD    1616.359594   0.391907  0.876330      618.674213
DVD    740.634027   0.303630  0.678938     1350.194514


As such, we have final results of 
$$
d_\text{CD} = 1616.4 \pm 0.4 \text{ nm}
$$

and 

$$
d_\text{DVD} = 740.6 \pm 0.3 \text{ nm}
$$

The standard DVD and CD grooves per mm [[2](https://nnci.net/sites/default/files/2020-02/CD_DVD_Diffraction%20grating_Part%201_SG_answers_0.pdf)]
 is about 1350 tracks per mm for a DVD and about 625 tracks per mm for a CD, this translates to


In [ ]:
g_dvd = 1350
g_cd = 625 
d_dvd = 1/g_dvd * 1e6 # transform from mm to nm
d_cd = 1/g_cd * 1e6

summary['d_actual_nm'] = [d_cd,d_dvd]
print(f'd_CD = {d_cd:.2f}')
print(f'd_DVD = {d_dvd:.2f}')

d_CD = 1600.00
d_DVD = 740.74


Which indicatest that the calculated result has a percent error $\epsilon$ of 

In [72]:
summary['eps'] = 100 * np.abs((summary['d_actual_nm'] - summary['d_mean_nm']) / summary['d_actual_nm'])
summary['eps'] = summary['eps'].apply(lambda x: f"{x:.2f}%")
print(summary[['d_actual_nm', 'd_mean_nm', 'eps']])

      d_actual_nm    d_mean_nm    eps
type                                 
CD    1600.000000  1616.359594  1.02%
DVD    740.740741   740.634027  0.01%


Thus, we were able to determine the grating distance in both DVD and CD with very little error using the grating equation